In [46]:
import numpy as np
import pandas as pd
import plotly.express as px
import http.client
import json

This notbook will look into various cryptocurrencys correlations during volatile and regular conditions.

In [47]:
def getData(symbol, start, end):
    conn = http.client.HTTPSConnection("data.messari.io")
    conn.request("GET", "/api/v1/assets/" + symbol + "/metrics/price/time-series?start=" + start + "&end=" + end + "&interval=1d")
    res = conn.getresponse()
    data = json.loads(res.read())
    return pd.DataFrame(data["data"]["values"],columns=["timestamp","open","high","low","close","volume"]).set_index("timestamp")    

In [48]:
# GET Dataframes
eth = getData('eth','2020-01-01','2022-05-01')
sol = getData('sol','2020-01-01','2022-05-01')
btc = getData('btc','2020-01-01','2022-05-01')

In [58]:
# CALCULATE log returns (daily)
eth['log_ret_eth'] = np.log(eth.close) - np.log(eth.close.shift(1))
sol['log_ret_sol'] = np.log(sol.close) - np.log(sol.close.shift(1))
btc['log_ret_btc'] = np.log(btc.close) - np.log(btc.close.shift(1))

In [67]:
compare_returns = pd.concat([eth,sol,btc],axis=1)
compare_returns = compare_returns[["log_ret_eth","log_ret_btc","log_ret_sol"]]
overallCorr = compare_returns.dropna().corr()
overallCorr

,log_ret_eth,log_ret_btc,log_ret_sol
log_ret_eth,1.000000,0.785318,0.528090
log_ret_btc,0.785318,1.000000,0.393188
log_ret_sol,0.528090,0.393188,1.000000


In [66]:
corrHeatmap = px.imshow(overallCorr)
corrHeatmap.show()

In [74]:
# During high vol?
# Defiing high vol as when btc return was > 0.1
highVolReturns = compare_returns[compare_returns["log_ret_btc"] > 0.1]
highVolCorr = highVolReturns.dropna().corr()
highVolCorr

,log_ret_eth,log_ret_btc,log_ret_sol
log_ret_eth,1.000000,-0.137566,0.765016
log_ret_btc,-0.137566,1.000000,-0.019143
log_ret_sol,0.765016,-0.019143,1.000000


In [75]:
corrHeatmapHV = px.imshow(highVolCorr)
corrHeatmapHV.show()